In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
key = 'SIR'
var_list = ['up1', 'up2', 'down1', 'down2']
city_list = ['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', 
             '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
               'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

lr = 1e-3
batch_size = 64
patience = 50
num_layers = 1
hidden_size = 16
dropout = 0.25
criterion = nn.MSELoss()
var_iterator = tqdm(var_list, position = 0)

trained_var = {}
var_num = 0
for var in var_iterator:
    var_iterator.set_description(f"var : {var}, ({var_num+1}/4)")
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)
    
    trained_data = {}

    city_iterator = tqdm(city_list, position = 1)
    for city in city_iterator:
        
        data = dic_files[city]
        trained_model = {}

        input_size = 3
        sequence_length = 60
        num_epochs = 10000

        df = Prepare_df.processing(data, 'Date', 'alpha')

        x = df.iloc[:, 0:]
        y = df.iloc[:,:1]

        ms = MinMaxScaler()
        ss = StandardScaler()

        ss.fit(x)
        ms.fit(y)

        for num_model in range(12):
            city_iterator.set_description(f"{var} - {city}, ({num_model+1}/12)")
            model_name = models_list[num_model]


            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 238, 60, 1, batch_size, 'mto')
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 225, 60, 14, batch_size, 'mtm')

            if num_model == 0:
                model = RNN(input_size = input_size,
                            hidden_size = hidden_size,
                            sequence_length = sequence_length,
                            num_layers = num_layers, 
                            dropout = dropout, 
                            device = device).to(device)

            elif num_model == 1:
                model = LSTM(input_size = input_size,
                             hidden_size = hidden_size,
                             sequence_length = sequence_length,
                             num_layers = num_layers, 
                             dropout = dropout, 
                             device = device).to(device)

            elif num_model == 2:
                model = GRU(input_size = input_size,
                            hidden_size = hidden_size,
                            sequence_length = sequence_length,
                            num_layers = num_layers, 
                            dropout = dropout, 
                            device = device).to(device)

            elif num_model == 3:
                model = BiRNN(input_size = input_size,
                              hidden_size = hidden_size,
                              sequence_length = sequence_length,
                              num_layers = num_layers, 
                              dropout = dropout, 
                              device = device).to(device)

            elif num_model == 4:
                model = BiLSTM(input_size = input_size,
                               hidden_size = hidden_size,
                               sequence_length = sequence_length,
                               num_layers = num_layers, 
                               dropout = dropout, 
                               device = device).to(device)

            elif num_model == 5:
                model = BiGRU(input_size = input_size,
                              hidden_size = hidden_size,
                              sequence_length = sequence_length,
                              num_layers = num_layers, 
                              dropout = dropout, 
                              device = device).to(device)

            elif num_model == 6:
                model = RNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

            elif num_model == 7:
                model = LSTM_encoder_decoder(input_size = input_size, 
                                             hidden_size = hidden_size,
                                             num_layers = num_layers, 
                                             dropout = dropout,
                                             device = device).to(device)

            elif num_model == 8:
                model = GRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

            elif num_model == 9:
                model = BiRNN_encoder_decoder(input_size = input_size, 
                                                hidden_size = hidden_size,
                                                num_layers = num_layers, 
                                                dropout = dropout,
                                                device = device).to(device)

            elif num_model == 10:
                model = BiLSTM_encoder_decoder(input_size = input_size, 
                                                  hidden_size = hidden_size,
                                                  num_layers = num_layers, 
                                                  dropout = dropout,
                                                  device = device).to(device)

            elif num_model == 11:
                model = BiGRU_encoder_decoder(input_size = input_size, 
                                                hidden_size = hidden_size,
                                                num_layers = num_layers, 
                                                dropout = dropout,
                                                device = device).to(device)            

            optimizer = Adam(model.parameters(), lr = lr)

            if num_model < 6:
                loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                              test_loader, 
                                                              model, 
                                                              criterion, 
                                                              optimizer, 
                                                              num_epochs, 
                                                              patience,
                                                              device)

                label_y, predicted = predict_mto(model, df, x_ss, y_ms)

                rmse_val = rmse(label_y[-14:], predicted[-14:])
                trained_model[models_list[num_model]] = [model, rmse_val, epoch, lr, patience, 
                                                         num_layers,hidden_size, dropout]


            else:
                loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                               test_loader, 
                                                               model, 
                                                               criterion, 
                                                               optimizer, 
                                                               num_epochs, 
                                                               patience,
                                                               14,
                                                               device)        

                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(model, df, x_ss, y_ms, 
                                                                                                14, 0.5, device)

                rmse_val = rmse(label_y[-14:], predicted[-14:])
                trained_model[models_list[num_model]] = [model, rmse_val, epoch, lr, patience, 
                                                         num_layers, hidden_size, dropout, criterion]

        trained_data[city] = trained_model
        
    trained_var[var] = trained_data    
    
    var_num += 1
        

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [3]:
for var in trained_var:
    for city in trained_var[var]:
        for name, values in trained_var[var][city].items():
            save_model(values[0].state_dict(), f"model/Stricgency/{key}/{var}/{city}/{var}_{city}_{name}.pth")
            save_hyperparameter(values[1:], f"hyperparameter/Stricgency/{key}/{var}/{city}/{var}_{city}_{name}.pkl")